In [8]:
#################################
#**  Change file name below  **

testfile_name = r"C:\Users\Rohit Varre\Desktop\MliS\ML lab\Assignment 2\testDatasetExample.xls"

#*********************************

<h2> Import all necessary packages <h2\>

In [9]:

import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif

from scipy import stats as st
from sklearn.preprocessing import PowerTransformer

import pickle

<h2> Reading the data <h2\>

In [10]:
test_df = pd.read_excel(testfile_name)

In [15]:
# load the imputer from disk
filename = 'saved_model/int_cols_imputer.sav'
sic = pickle.load(open(filename, 'rb'))

filename = 'saved_model/float_cols_imputer.sav'
sif = pickle.load(open(filename, 'rb'))

# load the transformer from disk
filename = 'saved_model/robust_scaler.sav'
robust_scaler = pickle.load(open(filename, 'rb'))

# load the scaler from disk
filename = 'saved_model/min_max_scaler.sav'
min_max_scaler = pickle.load(open(filename, 'rb'))

# Continuous columns list
float_cols = pickle.load(open('saved_model/continuous_features.pkl', 'rb'))['continuous_features']

# Categorical columns
cat_cols = pickle.load(open('saved_model/categorical_features.pkl', 'rb'))['categorical_features']

## Loadin the model
model = tf.keras.models.load_model('saved_model/best_model_aucc82')


<h2> Data Preprocessing <h2/>

<h3> Filling missing values <h3/>

In [16]:
df_cats = test_df[cat_cols]
df_floats = test_df[float_cols]

<h3> Merging features <h3/>

In [25]:
df_cats = pd.DataFrame(sic.transform(df_cats),columns=df_cats.columns)
df_floats = pd.DataFrame(sif.transform(df_floats),columns=df_floats.columns)
X = pd.concat([df_cats.reset_index(drop=True),df_floats.reset_index(drop=True)],axis=1)

<h3> Scaling the features <h3/>

In [26]:
X = pd.DataFrame(robust_scaler.transform(X),columns=X.columns)
X = pd.DataFrame(min_max_scaler.transform(X),columns=X.columns)

C:\Users\Rohit Varre\miniconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


<h2> Model prediction <h2/>

In [19]:
predictions = model.predict(X)

In [21]:
preds_df = pd.DataFrame(zip(test_df["ID"].to_list(),
                            list(np.squeeze(predictions))),
                            columns=(["ID","RFS"]))

In [23]:
preds_df.head()

,ID,RFS
0,TRG002728,58.606575
1,TRG002649,58.606575
2,TRG002628,58.606575


In [24]:
preds_df.to_csv('pcr_predictions.csv',index=False)